# Notebook Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Constants

In [2]:
vocab_size = 2500

training_data_file = 'SpamData/02_Training/Train-data.txt'
testing_data_file = 'SpamData/03_Testing/Test-data.txt'

token_spam_prob_file = 'SpamData/03_Testing/prob-spam.txt'
toekn_ham_prob_file = 'SpamData/03_Testing/prob-ham.txt'
toekn_all_prob_file = 'SpamData/03_Testing/prob-all-tokens.txt'

test_feature_matrix = 'SpamData/03_Testing/test-features.txt'
test_target_file = 'SpamData/03_Testing/test-target.txt'

## Read and Load Features from .txt  files into numPy Array

In [3]:
sparse_train_data = np.loadtxt(training_data_file, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(testing_data_file, delimiter=' ', dtype=int)

In [4]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
print("Nr of rows in training file: ", sparse_train_data.shape[0])
print("Nr of rows in testing file: ", sparse_test_data.shape[0])

Nr of rows in training file:  258447
Nr of rows in testing file:  117827


In [6]:
print('Nr of emails Training filel ', np.unique(sparse_train_data[:, 0]).size)
print('Nr of emails Testing filel ', np.unique(sparse_test_data[:, 0]).size)

Nr of emails Training filel  4013
Nr of emails Testing filel  1724


### How to Create an Empty Dataframe

In [7]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocab_size))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [8]:
len(column_names)

2502

In [9]:
index_names = np.unique(sparse_train_data[: ,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795], shape=(4013,))

In [40]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)
pd.set_option('future.no_silent_downcasting', True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a Full Matrix from sparse matrix


In [46]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocab_size))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [12]:
%%time
full_train_data = make_full_matrix(sparse_train_data, vocab_size)

CPU times: total: 14.2 s
Wall time: 17.2 s


In [13]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculate the Probability of Spam

In [14]:
# Total number of emails
full_train_data.CATEGORY.size

4013

In [15]:
# Total number of spam emails
full_train_data.CATEGORY.sum()

np.int64(1248)

In [16]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probablity of spam is ', prob_spam)

Probablity of spam is  0.310989284824321


In [17]:
features =  full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
email_words = features.sum(axis=1)
email_words

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 4013, dtype: object

In [19]:
total_wr = email_words.sum()
total_wr

np.int64(429464)

## Number of Tokens in Spam and Non-Spam

In [20]:
spam_words = email_words[full_train_data.CATEGORY == 1]
spam_words.shape

(1248,)

In [21]:
non_spam_words = email_words[full_train_data.CATEGORY == 0]
non_spam_words.shape

(2765,)

In [22]:
spam_wc = spam_words.sum()
spam_wc

np.int64(176378)

In [23]:
non_spam_wc = non_spam_words.sum()
non_spam_wc

np.int64(253086)

In [24]:
email_words.shape[0] - spam_words.shape[0] - non_spam_words.shape[0]

0

In [25]:
print('Average number of words in spam emails', round(spam_wc/spam_words.shape[0]))
print('Average number of words in non-emails', round(non_spam_wc/non_spam_words.shape[0]))

Average number of words in spam emails 141
Average number of words in non-emails 92


### Summing the number of tokens Occuring in Spam


In [26]:
features.shape

(4013, 2500)

In [27]:
tokens = features.loc[full_train_data.CATEGORY == 1]
tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
summed_spam_tokens = tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

In [29]:
ham_tokens = features.loc[full_train_data.CATEGORY == 0]
ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
summed_ham_tokens = ham_tokens.sum(axis = 0) + 1
summed_ham_tokens.shape

(2500,)

## P(Token l Spam) - Probablity that a Token Occurs given the email spam

In [31]:
prob_spam_tokens = summed_spam_tokens / (spam_wc + vocab_size)
prob_spam_tokens

0       0.012181
1       0.005227
2       0.006804
3       0.011304
4       0.006815
          ...   
2495    0.000045
2496    0.000056
2497    0.000095
2498    0.000045
2499    0.000006
Length: 2500, dtype: object

In [32]:
prob_spam_tokens.sum()

np.float64(0.9999999999999944)

## P(Token l Non-Spam) - Probablity that a Token Occurs given the email Non-spam

In [33]:
prob_ham_tokens = summed_ham_tokens / (non_spam_wc + vocab_size)
prob_ham_tokens

0       0.021457
1       0.010137
2       0.008017
3        0.00367
4       0.006307
          ...   
2495    0.000106
2496    0.000102
2497    0.000078
2498    0.000094
2499    0.000113
Length: 2500, dtype: object

## P(Tokens) - Probability that a token Occurs

In [34]:
prob_tokens_all = features.sum(axis= 0) / total_wr
prob_tokens_all.sum()

np.float64(0.9999999999999992)

##  Saving

In [35]:
np.savetxt(token_spam_prob_file, prob_spam_tokens)
np.savetxt(toekn_ham_prob_file, prob_ham_tokens)
np.savetxt(toekn_all_prob_file, prob_tokens_all)

## Prepare Test DAta



In [44]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)
pd.set_option('future.no_silent_downcasting', True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=vocab_size)

CPU times: total: 6.44 s
Wall time: 6.78 s


In [49]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [54]:
sparse_test_data.shape

(117827, 4)

In [52]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [53]:
np.savetxt(test_target_file, y_test)
np.savetxt(test_feature_matrix, x_test)